## Code to pull pixel brightness values along centerlines

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import matplotlib.image as mpimg
import cv2
import pandas as pd    
import PIL

csvpaths = '/home/jukes/Documents/Sample_glaciers/'
basepath = '/media/jukes/jukes1/LS8aws/'
massorsize = "mass"

Grab centerline info and BoxIDs

In [ ]:
centerline_df = pd.read_csv(csvpaths+'Boxes_coords_pathrows.csv', sep=',', dtype=str, header=0)
centerline_df = centerline_df.set_index('BoxID')
print(centerline_df.shape)

BoxIDs = list(centerline_df.index)
print(BoxIDs)

centerline_df

In [ ]:
#make lists to store image data and grab image files
ratios = []
imagenames = []

for BoxID in BoxIDs:
#     if BoxID == '033':
        imagepath = basepath+"Box"+BoxID+"/resized/"
        #image path
        imgfiles = os.listdir(imagepath)

        #grab terminus box raster
        tbox = mpimg.imread(csvpaths+"Box"+BoxID+"/rotated_images/final/crop_Box"+BoxID+"_raster_cut.png")

        #grab centerline points
        xmin = float(centerline_df.loc[BoxID]['midpoint_x'])
        xmax = np.max(np.array(list(set(np.nonzero(tbox)[0]))))
        c_x = np.linspace(xmin, xmax, int(xmax-xmin))
        c_y = float(centerline_df.loc[BoxID]['cline_slope'])*c_x + float(centerline_df.loc[BoxID]['cline_intercept'])

        #load in each image and grab the pixel values indexed by the centerline points:
        for img in imgfiles:
            if img.endswith(BoxID+'.png'):
                imagename_pgm = img[:-4]+'.pgm'
                #load in the image using cv2
                image = cv2.imread(imagepath+img,-1)
                imageplt = mpimg.imread(imagepath+img)

                pixelvalues = []

                #loop through all centerline coordinates
                for i in range(0, len(c_x)):
                    x = int(c_x[i])
                    y = int(c_y[i])
                    pixval = '%.3f' % image[y][x]
                    pixelvalues.append(float(pixval))
                    
#                     print(x, y, pixval)

#                 #grab the first and last third
                third = int(len(pixelvalues)/3)
                third1 = pixelvalues[:third]
                third3 = pixelvalues[-third:]
#                 print(third1)
#                 print(third3)

                #grab the average of the first and average of the last to compare
                avgthird1 = float(np.average(np.array(third1)))
                avgthird3 = float(np.average(np.array(third3)))
                ratio = "%.2f" % (avgthird1/avgthird3)
                
                ratios.append(ratio)
                imagenames.append(imagename_pgm)
# #                 print('Ratio 1st/3rd:', ratio)
#                 print(img)

#                 plt.figure(figsize=(20,10))
#                 imgplt_trim = plt.imshow(cv2.cvtColor(imageplt, cv2.COLOR_BGR2RGB))
#                 plt.plot(c_x, c_y, '--w', linewidth=1.5, alpha=0.3)
#                 plt.text(int(0.02*image.shape[0]), int(0.035*image.shape[1]), ratio
#                          , fontsize=16, color='w')
#                 plt.show()
pixels_df = pd.DataFrame(list(zip(imagenames, ratios)))
pixels_df

In [ ]:
#write to txt file
pixels_df.to_csv(path_or_buf=csvpaths+'Pixelratios.txt', sep=' ', index=False, header=False)